In [2]:
import pandas as pd

In [57]:
df=pd.read_csv("D:\ABIDEII_Composite_Phenotypic.csv")

In [18]:
df.head()

,SITE_ID,SUB_ID,NDAR_GUID,DX_GROUP,PDD_DSM_IV_TR,ASD_DSM_5,AGE_AT_SCAN,SEX,HANDEDNESS_CATEGORY,HANDEDNESS_SCORES,...,ADI_R_C3_TOTAL,ADI_R_C4_REPETITIVE_USE_OBJECTS,ADI_R_C4_HIGHER,ADI_R_C4_UNUSUAL_SENSORY_INTERESTS,ADI_R_C4_TOTAL,ADI_R_D_AGE_PARENT_NOTICED,ADI_R_D_AGE_FIRST_SINGLE_WORDS,ADI_R_D_AGE_FIRST_PHRASES,ADI_R_D_AGE_WHEN_ABNORMALITY,ADI_R_D_INTERVIEWER_JUDGMENT
0,ABIDEII-BNI_1,29006,NaN,1,NaN,NaN,48.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ABIDEII-BNI_1,29007,NaN,1,NaN,NaN,41.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABIDEII-BNI_1,29008,NaN,1,NaN,NaN,59.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABIDEII-BNI_1,29009,NaN,1,NaN,NaN,57.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABIDEII-BNI_1,29010,NaN,1,NaN,NaN,45.0,1,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.columns

Index(['SITE_ID', 'SUB_ID', 'NDAR_GUID', 'DX_GROUP', 'PDD_DSM_IV_TR',
       'ASD_DSM_5', 'AGE_AT_SCAN ', 'SEX', 'HANDEDNESS_CATEGORY',
       'HANDEDNESS_SCORES',
       ...
       'ADI_R_C3_TOTAL', 'ADI_R_C4_REPETITIVE_USE_OBJECTS', 'ADI_R_C4_HIGHER',
       'ADI_R_C4_UNUSUAL_SENSORY_INTERESTS', 'ADI_R_C4_TOTAL',
       'ADI_R_D_AGE_PARENT_NOTICED', 'ADI_R_D_AGE_FIRST_SINGLE_WORDS',
       'ADI_R_D_AGE_FIRST_PHRASES', 'ADI_R_D_AGE_WHEN_ABNORMALITY',
       'ADI_R_D_INTERVIEWER_JUDGMENT'],
      dtype='object', length=348)

In [20]:
df = df[["AGE_AT_SCAN ", "SEX", "FIQ","HANDEDNESS_CATEGORY"]]
df.rename(columns={"AGE_AT_SCAN": "Age"}, inplace=True)
# target = {1: "Autism", 2: "NonAutism"}
gender = {1: "Male", 2: "Female"}
handness = {1: "right handed", 2:"left handed", 3:"mixed handed"}
# 1=right handed; 2=left handed; 3=mixed handed 
# df["Target"] = df["DX_GROUP"].map(target)
df["SEX"] = df["SEX"].map(gender)
df["HANDEDNESS_CATEGORY"]= df["HANDEDNESS_CATEGORY"].map(handness)
# df.drop("DX_GROUP", axis=1, inplace=True)
df.head()


,AGE_AT_SCAN,SEX,FIQ,HANDEDNESS_CATEGORY
0,48.0,Male,131.0,right handed
1,41.0,Male,110.0,right handed
2,59.0,Male,117.0,right handed
3,57.0,Male,114.0,right handed
4,45.0,Male,109.0,right handed


In [21]:
df.shape

(1113, 4)

In [6]:
!pip install openai
!pip install openai==0.28

  Attempting uninstall: openai
    Found existing installation: openai 1.58.1
    Uninstalling openai-1.58.1:
      Successfully uninstalled openai-1.58.1


In [22]:
import openai

def analyze_patient_data(data_sample, api_base="http://localhost:1234/v1", api_key="not-needed", model="llama-3.2-3b-instruct"):
    """
    Analyze patient metadata and format it into a single English sentence.
    
    Parameters:
        data_sample (str): The sample data to analyze.
        api_base (str): The base URL for the OpenAI API.
        api_key (str): The API key for authentication.
        model (str): The model name (currently unused).
    
    Returns:
        str: The formatted analysis response from the model.
    """
    # Set the base URL and API key for the OpenAI client
    openai.api_base = api_base
    openai.api_key = api_key

    # Create a chat completion
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "user", 
                "content": f"""Analyze the provided data based on the patient's metadata and format it into a single English sentence. Ensure the response includes all four input fields: Age, SEX, FIQ, and Target. The output as a plain string with no preamble in English response with a single sentence: ({data_sample})"""
            }
        ],
        temperature=0.7,
    )

    # Return the chatbot's response
    return completion.choices[0].message.content

# Example usage
sample_data = df.iloc[0].to_dict()
print(sample_data)
response = analyze_patient_data(sample_data)
print(response)


{'AGE_AT_SCAN ': 48.0, 'SEX': 'Male', 'FIQ': 131.0, 'HANDEDNESS_CATEGORY': 'right handed'}
A 48-year-old male with an FIQ score of 131 and right-handed handedness is the target patient for analysis.


In [23]:
import openai

def analyze_patient_data(data_sample, api_base="http://localhost:1234/v1", api_key="not-needed", model="llama-3.2-3b-instruct"):
    """
    Analyze patient metadata and format it into a single English sentence.
    
    Parameters:
        data_sample (str): The sample data to analyze.
        api_base (str): The base URL for the OpenAI API.
        api_key (str): The API key for authentication.
        model (str): The model name (currently unused).
    
    Returns:
        str: The formatted analysis response from the model.
    """
    # Set the base URL and API key for the OpenAI client
    openai.api_base = api_base
    openai.api_key = api_key

    # Create a chat completion
    completion = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "user", 
                "content": f"""Analyze the provided data based on the patient's metadata and format it into a single English sentence. Ensure the response includes all four input fields: Age, SEX, FIQ, and Target. The output as a plain string with no preamble in English response with a single sentence: ({data_sample})"""
            }
        ],
        temperature=0.7,
    )

    # Return the chatbot's response
    print("--"*10)
    print(completion.choices[0].message.content)
    print(type(completion.choices[0].message.content))
    print("--"*10)
    return completion.choices[0].message.content

# Example usage
sample_data = df.iloc[0].to_dict()
print(sample_data)
response = analyze_patient_data(sample_data)
# print(response)


{'AGE_AT_SCAN ': 48.0, 'SEX': 'Male', 'FIQ': 131.0, 'HANDEDNESS_CATEGORY': 'right handed'}
--------------------
A 48-year-old right-handed male patient has an FIQ score of 131.0.
<class 'str'>
--------------------


In [16]:
import openai
# FInal format Phani
def analyze_patient_data(data_sample, api_base="http://localhost:1234/v1", api_key="not-needed", model="llama-3.2-3b-instruct"):
    """
    Analyze patient metadata and format it into a single English sentence.
    
    Parameters:
        data_sample (dict): The sample data to analyze.
        api_base (str): The base URL for the OpenAI API.
        api_key (str): The API key for authentication.
        model (str): The model name (currently unused).
    
    Returns:
        str: The formatted analysis response from the model.
    """
    # Set the base URL and API key for the OpenAI client
    openai.api_base = api_base
    openai.api_key = api_key

    while True:  # Keep attempting until a string is returned
        # Create a chat completion
        completion = openai.ChatCompletion.create(
            model=model,
            messages=[
                {
                    "role": "user", 
                    "content": f"""Analyze the provided data based on the patient's metadata and format it into a single English sentence. Ensure the response includes all four input fields: Age, SEX, FIQ, and Target. The output as a plain string with no preamble in English response with a single sentence: ({data_sample})"""
                }
            ],
            temperature=0.7,
        )

        # Extract the response
        response_content = completion.choices[0].message.content
        
        # Check if the response is a string
        if isinstance(response_content, str):
            print(response_content)
            return response_content  # Return the string response
        else:
            print("Response was not a string. Retrying...")

# Example usage
# Replace `df.iloc[0].to_dict()` with an example dictionary for testing if `df` is undefined.
sample_data = {"Age": 55, "SEX": "Male", "FIQ": 126, "Target": "Treatment for Autism"}  
response = analyze_patient_data(sample_data)

# print("Final Response:")
# print(response)


A 55-year-old male patient with an FIQ score of 126 is seeking treatment for their autism.


In [24]:
sen_eng_data = []
for i in range(len(df)):
    sample_data = df.iloc[i].to_dict()
    print(sample_data)
    sen_eng_data.append(analyze_patient_data(sample_data))

{'AGE_AT_SCAN ': 48.0, 'SEX': 'Male', 'FIQ': 131.0, 'HANDEDNESS_CATEGORY': 'right handed'}
--------------------
A 48-year-old male with an FIQ score of 131 and right-handed handedness is the target patient.
<class 'str'>
--------------------
{'AGE_AT_SCAN ': 41.0, 'SEX': 'Male', 'FIQ': 110.0, 'HANDEDNESS_CATEGORY': 'right handed'}
--------------------
A 41-year-old male right-handed patient with an FIQ score of 110 and Target as the provided input fields.
<class 'str'>
--------------------
{'AGE_AT_SCAN ': 59.0, 'SEX': 'Male', 'FIQ': 117.0, 'HANDEDNESS_CATEGORY': 'right handed'}
--------------------
The patient, a 59-year-old right-handed male, has an FIQ score of 117.0 as their target in the dataset.
<class 'str'>
--------------------
{'AGE_AT_SCAN ': 57.0, 'SEX': 'Male', 'FIQ': 114.0, 'HANDEDNESS_CATEGORY': 'right handed'}
--------------------
The patient is a 57-year-old right-handed male with an FSIQ score of 114 who is the target for analysis.
<class 'str'>
--------------------
{'

In [25]:
df['Caption'] = sen_eng_data
df

,AGE_AT_SCAN,SEX,FIQ,HANDEDNESS_CATEGORY,Caption
0,48.0000,Male,131.0,right handed,A 48-year-old male with an FIQ score of 131 an...
1,41.0000,Male,110.0,right handed,A 41-year-old male right-handed patient with a...
2,59.0000,Male,117.0,right handed,"The patient, a 59-year-old right-handed male, ..."
3,57.0000,Male,114.0,right handed,The patient is a 57-year-old right-handed male...
4,45.0000,Male,109.0,right handed,A 45-year-old male with an FIQ score of 109 an...
...,...,...,...,...,...
1108,36.1451,Male,115.0,NaN,The 36-year-old male patient has a FIQ score o...
1109,11.7892,Male,99.0,NaN,The patient is an 11-year-old male with a high...
1110,23.2909,Male,123.0,right handed,The patient is a 23-year-old right-handed male...
1111,32.6242,Male,86.0,NaN,A 32-year-old male with an FIQ score of 86 is ...


In [28]:
df.to_csv('abide2.csv')

In [26]:
with open("cap3.txt","a") as f:
    for i in sen_eng_data:
        f.write(i+"\n")
    
        

In [27]:
with open("cap3.txt","r",encoding="utf-8") as d:
    data = d.readlines()
data = [i.strip() for i in data]
data

['A 48-year-old male with an FIQ score of 131 and right-handed handedness is the target patient.',
 'A 41-year-old male right-handed patient with an FIQ score of 110 and Target as the provided input fields.',
 'The patient, a 59-year-old right-handed male, has an FIQ score of 117.0 as their target in the dataset.',
 'The patient is a 57-year-old right-handed male with an FSIQ score of 114 who is the target for analysis.',
 'A 45-year-old male with an FIQ score of 109 and right-handed handedness is the target patient.',
 'The 62-year-old male patient with an FIQ score of 114 and right-handed dominance has {Target} as their target in the dataset.',
 'The 51-year-old male patient with an FIQ score of 93 and right-handed dominance has Target as the output variable.',
 'The patient, a 47-year-old male right-handed individual, presents with an FIQ score of 102.0.',
 'The 55-year-old right-handed male patient has a FIQ score of 99, indicating the severity and impact on their health.',
 'The 5

In [49]:
df1['subject']

0       50002
1       50003
2       50004
3       50005
4       50006
        ...  
1095    51583
1096    51584
1097    51585
1098    51606
1099    51607
Name: subject, Length: 1100, dtype: int64

In [37]:
df1.columns

Index(['SITE_ID', 'SUB_ID', 'NDAR_GUID', 'DX_GROUP', 'PDD_DSM_IV_TR',
       'ASD_DSM_5', 'AGE_AT_SCAN ', 'SEX', 'HANDEDNESS_CATEGORY',
       'HANDEDNESS_SCORES',
       ...
       'ADI_R_C3_TOTAL', 'ADI_R_C4_REPETITIVE_USE_OBJECTS', 'ADI_R_C4_HIGHER',
       'ADI_R_C4_UNUSUAL_SENSORY_INTERESTS', 'ADI_R_C4_TOTAL',
       'ADI_R_D_AGE_PARENT_NOTICED', 'ADI_R_D_AGE_FIRST_SINGLE_WORDS',
       'ADI_R_D_AGE_FIRST_PHRASES', 'ADI_R_D_AGE_WHEN_ABNORMALITY',
       'ADI_R_D_INTERVIEWER_JUDGMENT'],
      dtype='object', length=348)

In [39]:
df1=pd.read_csv("D:\ABIDEII_Composite_Phenotypic.csv")
df1 = df1[["SITE_ID","SUB_ID"]]
df1['SUB_ID'].astype(str)
df1.head()

,SITE_ID,SUB_ID
0,ABIDEII-BNI_1,29006
1,ABIDEII-BNI_1,29007
2,ABIDEII-BNI_1,29008
3,ABIDEII-BNI_1,29009
4,ABIDEII-BNI_1,29010


In [43]:
# df1['path'] = df1['SITE_ID'] + "_" + str( df1['subject'])
# df1.head()

# [(i,j) for i,j in (df1['SITE_ID'],str(df1['subject']))]\

paths = []
for i , j in zip(df1['SITE_ID'], df1['SUB_ID']):
    paths.append(i+"_"+str(j))

In [50]:
paths

['ABIDEII-BNI_1_29006',
 'ABIDEII-BNI_1_29007',
 'ABIDEII-BNI_1_29008',
 'ABIDEII-BNI_1_29009',
 'ABIDEII-BNI_1_29010',
 'ABIDEII-BNI_1_29012',
 'ABIDEII-BNI_1_29013',
 'ABIDEII-BNI_1_29015',
 'ABIDEII-BNI_1_29017',
 'ABIDEII-BNI_1_29025',
 'ABIDEII-BNI_1_29026',
 'ABIDEII-BNI_1_29027',
 'ABIDEII-BNI_1_29028',
 'ABIDEII-BNI_1_29029',
 'ABIDEII-BNI_1_29030',
 'ABIDEII-BNI_1_29031',
 'ABIDEII-BNI_1_29037',
 'ABIDEII-BNI_1_29039',
 'ABIDEII-BNI_1_29041',
 'ABIDEII-BNI_1_29042',
 'ABIDEII-BNI_1_29043',
 'ABIDEII-BNI_1_29046',
 'ABIDEII-BNI_1_29052',
 'ABIDEII-BNI_1_29053',
 'ABIDEII-BNI_1_29055',
 'ABIDEII-BNI_1_30144',
 'ABIDEII-BNI_1_30148',
 'ABIDEII-BNI_1_30150',
 'ABIDEII-BNI_1_30151',
 'ABIDEII-BNI_1_29011',
 'ABIDEII-BNI_1_29014',
 'ABIDEII-BNI_1_29016',
 'ABIDEII-BNI_1_29018',
 'ABIDEII-BNI_1_29019',
 'ABIDEII-BNI_1_29020',
 'ABIDEII-BNI_1_29021',
 'ABIDEII-BNI_1_29022',
 'ABIDEII-BNI_1_29023',
 'ABIDEII-BNI_1_29024',
 'ABIDEII-BNI_1_29032',
 'ABIDEII-BNI_1_29033',
 'ABIDEII-BNI_1_

In [46]:
new=pd.read_csv('abide2.csv')

In [60]:
new['Path'] = paths
new['Caption'] = data
new['Target'] =df['DX_GROUP']

In [62]:
new.to_csv('abide2_label.csv')

In [49]:
paths=new['Path']

KeyError: 'Path'

In [58]:
df1['Path'] = paths
df1['Caption'] = data
df1['Target'] =df['DX_GROUP']
df1.head()

,SITE_ID,SUB_ID,Path,Caption,Target
0,ABIDEII-BNI_1,29006,ABIDEII-BNI_1_29006,A 48-year-old male with an FIQ score of 131 an...,1
1,ABIDEII-BNI_1,29007,ABIDEII-BNI_1_29007,A 41-year-old male right-handed patient with a...,1
2,ABIDEII-BNI_1,29008,ABIDEII-BNI_1_29008,"The patient, a 59-year-old right-handed male, ...",1
3,ABIDEII-BNI_1,29009,ABIDEII-BNI_1_29009,The patient is a 57-year-old right-handed male...,1
4,ABIDEII-BNI_1,29010,ABIDEII-BNI_1_29010,A 45-year-old male with an FIQ score of 109 an...,1


In [26]:
df.to_csv('Label_Data1.csv')

In [93]:
import cv2

In [94]:
# import os
# import io
# from PIL import Image
# import pandas as pd
# from datasets import Dataset

# # Load the DataFrame (replace with your actual DataFrame loading method)
# df = pd.DataFrame({
#     'Age': [55.4, 22.9, 39.2, 22.8, 34.6],
#     'SEX': ['Male', 'Male', 'Male', 'Male', 'Female'],
#     'FIQ': [126, 107, 93, 106, 133],
#     'Target': ['Autism', 'Autism', 'Autism', 'Autism', 'Autism'],
#     'Caption': [
#         "The patient is a 55-year-old male...",
#         "The patient is 22 years and nine months...",
#         "The patient is a 39-year-old male...",
#         "The patient is a 23-year-old male...",
#         "The patient is a 34.6-year-old female..."
#     ],
#     'Path': ['CALTECH_51456', 'CALTECH_51457', 'CALTECH_51458', 'CALTECH_51459', 'CALTECH_51460']
# })

# # Define the folders
# autism_folder = "Autism"
# non_autism_folder = "NonAutism"

# # Helper function to load images and convert them to bytes
# def load_image_as_bytes(file_name, folders):
#     for folder in folders:
#         for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
#             full_path = os.path.join(folder, file_name + ext)
#             if os.path.exists(full_path):
#                 print(f"Image found: {full_path}")  # Debugging log
#                 with open(full_path, "rb") as f:  # Read as binary
#                     return f.read()
#     print(f"Image not found for {file_name}")  # Debugging log
#     return None

# # Load images and create dataset components
# image_data = []
# labels = []
# captions = []

# for _, row in df.iterrows():
#     img_bytes = load_image_as_bytes(row['Path'], [autism_folder, non_autism_folder])
#     if img_bytes:  # Only append if the image is successfully loaded
#         image_data.append(img_bytes)  # Append byte-encoded images
#         labels.append(row['Target'])
#         captions.append(row['Caption'])

# # Check if any images were loaded
# if not image_data:
#     print("No images were loaded. Check the file paths and extensions.")
# else:
#     # Create Hugging Face Dataset with image, label, and caption
#     data_dict = {
#         'image': image_data,
#         'label': labels,
#         'caption': captions
#     }

#     dataset = Dataset.from_dict(data_dict)

#     # Convert dataset to Pandas DataFrame for verification
#     df_result = dataset.to_pandas()
#     display(df_result)

#     # Push dataset to Hugging Face Hub
#     # dataset.push_to_hub("your-username/your-dataset-name")


In [95]:
# import os
# from PIL import Image
# import io

# # Define the folders
# autism_folder = "Autism"
# non_autism_folder = "NonAutism"

# # Helper function to load image bytes
# def load_image_as_bytes(file_name, folders):
#     for folder in folders:
#         for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
#             full_path = os.path.join(folder, file_name + ext)
#             if os.path.exists(full_path):
#                 print(f"Image found: {full_path}")  # Debugging log
#                 with open(full_path, "rb") as f:  # Read as binary
#                     return f.read()
#     print(f"Image not found for {file_name}")  # Debugging log
#     return None

# # Read the first row of the dataframe
# first_row = df.iloc[4]
# image_path = first_row['Path']
# folders = [autism_folder, non_autism_folder]

# # Load the image bytes
# image_bytes = load_image_as_bytes(image_path, folders)

# # Display the image
# if image_bytes:
#     image = Image.open(io.BytesIO(image_bytes))  # Convert bytes back to an image
#     image.show()  # Show the image
#     print(f"Image Bytes Length: {len(image_bytes)}")  # Display the length of bytes
# else:
#     print("Image could not be loaded. Check the file path or extensions.")


In [13]:
df.shape

(38, 6)

In [59]:
!pip install datasets


  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.64.0
    Uninstalling tqdm-4.64.0:
      Successfully uninstalled tqdm-4.64.0
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
tensorflow-intel 2.16.2 requires flatbuffers>=23.5.26, but you have flatbuffers 1.12 which is incompatible.
tensorflow-intel 2.16.2 requires keras>=3.0.0, but you have keras 2.9.0 which is incompatible.
tensorflow-intel 2.16.2 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.19.6 which is incompatible.
tensorflow-intel 2.16.2 requires tensorboard<2.17,>=2.16, but you have tensorboard 2.9.1 which is incompatible.
qiskit-ibmq-provider 0.20.2 requires numpy<1.24, but you have numpy 1.26.4 which is incompatible.


    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2022.2.0
    Uninstalling fsspec-2022.2.0:
      Successfully uninstalled fsspec-2022.2.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


In [33]:
dfg=pd.read_csv('temp.csv')

In [80]:
df

,Unnamed: 0,Age,SEX,FIQ,EYE_STATUS_AT_SCAN,Target,Caption,Path
0,0,16.77,Male,103.0,closed,Autism,The patient is a 16-year-old male with an FIQ ...,PITT_50002
1,1,24.45,Male,124.0,closed,Autism,A 24-year-old male with an FIQ score of 124 an...,PITT_50003
2,2,19.09,Male,113.0,closed,Autism,The 19-year-old male patient with an FIQ score...,PITT_50004
3,3,13.73,Female,119.0,closed,Autism,The female patient aged 13.73 with an FIQ scor...,PITT_50005
4,4,13.37,Male,109.0,closed,Autism,The 13-year-old male patient with an FIQ score...,PITT_50006
...,...,...,...,...,...,...,...,...
1095,1095,35.00,Male,-9999.0,closed,Autism,"The patient is a male, aged 35, diagnosed with...",SBL_51583
1096,1096,49.00,Male,-9999.0,closed,Autism,The patient is a 49-year-old male with an FIQ ...,SBL_51584
1097,1097,27.00,Male,-9999.0,closed,Autism,"The patient, a 27-year-old male with an unrepo...",SBL_51585
1098,1098,29.00,Female,-9999.0,closed,Autism,The patient is a 29-year-old female with an FI...,MAX_MUN_a_51606


In [35]:
import os
import io
from PIL import Image
import pandas as pd
from datasets import Dataset

# Define the folders
autism_folder = "sMRI_Data/Autism"
non_autism_folder = "sMRI_Data/NonAutism"

# Function to load images and convert them to bytes
def load_image_as_bytes(file_name, folders):
    for folder in folders:
        for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
            full_path = os.path.join(folder, file_name + ext)
            if os.path.exists(full_path):
                with open(full_path, "rb") as f:  # Read as binary
                    return f.read()
    return None

# Function to process the dataset
def process_dataframe(df, folders):
    image_data = []
    labels = []
    captions = []
    ages = []
    sexes = []
    fiqs = []
    eye_statuses = []

    for _, row in df.iterrows():
        img_bytes = load_image_as_bytes(row['Path'], folders)
        if img_bytes:  # Only append if the image is successfully loaded
            image_data.append(img_bytes)
            labels.append(row['Target'])
            captions.append(row['Caption'])
            ages.append(row['Age'])
            sexes.append(row['SEX'])
            fiqs.append(row['FIQ'])
            eye_statuses.append(row['EYE_STATUS_AT_SCAN'])
        else:
            print(f"Image not found: {row['Path']}")

    # Check lengths of all columns
    data_lengths = {
        "image": len(image_data),
        "label": len(labels),
        "caption": len(captions),
        "age": len(ages),
        "sex": len(sexes),
        "fiq": len(fiqs),
        "eye_status": len(eye_statuses)
    }

    for column, length in data_lengths.items():
        print(f"Length of {column}: {length}")

    # Ensure all columns have the same length
    if len(set(data_lengths.values())) > 1:
        print("Error: Column lengths are inconsistent. Dataset cannot be created.")
        return None

    # Create Hugging Face Dataset
    data_dict = {
        'image': image_data,
        'label': labels,
        'caption': captions,
        'age': ages,
        'sex': sexes,
        'fiq': fiqs,
        'eye_status': eye_statuses
    }
    return Dataset.from_dict(data_dict)

# Function to display an image by index
def display_image(dataset, index):
    if 0 <= index < len(dataset):
        image_bytes = dataset[index]['image']
        label = dataset[index]['label']
        caption = dataset[index]['caption']
        age = dataset[index]['age']
        sex = dataset[index]['sex']
        fiq = dataset[index]['fiq']
        eye_status = dataset[index]['eye_status']

        # Convert bytes back to an image
        image = Image.open(io.BytesIO(image_bytes))
        image.show()

        # Print metadata
        print(f"Label: {label}")
        print(f"Caption: {caption}")
        print(f"Age: {age}")
        print(f"Sex: {sex}")
        print(f"FIQ: {fiq}")
        print(f"Eye Status: {eye_status}")
    else:
        print(f"Invalid index: {index}. Dataset contains {len(dataset)} entries.")

# Example usage
# Assuming df is already loaded with the columns: Age, SEX, FIQ, EYE_STATUS_AT_SCAN, Target, Caption, Path
# df = pd.read_csv("path_to_your_csv_file.csv")

# Process the dataset
dataset = process_dataframe(df, [autism_folder, non_autism_folder])

# Display the first image for verification
if dataset:
    display_image(dataset, 10)  # Adjust index as needed


Length of image: 1100
Length of label: 1100
Length of caption: 1100
Length of age: 1100
Length of sex: 1100
Length of fiq: 1100
Length of eye_status: 1100
Label: Autism
Caption: The patient is a 21-year-old male with an FIQ score of 128 and no eye status at scan.
Age: 21.48
Sex: Male
FIQ: 128.0
Eye Status: closed


In [27]:
import os
import io
from PIL import Image
import pandas as pd
from datasets import Dataset



# Define the folders
autism_folder = "sMRI_Data/Autism"
non_autism_folder = "sMRI_Data/NonAutism"

# Function to load images and convert them to bytes
def load_image_as_bytes(file_name, folders):
    for folder in folders:
        for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
            full_path = os.path.join(folder, file_name + ext)
            if os.path.exists(full_path):
                with open(full_path, "rb") as f:  # Read as binary
                    return f.read()
    return None

# Function to process the dataset
def process_dataframe(df, folders):
    image_data = []
    labels = []
    captions = []

    for _, row in df.iterrows():
        img_bytes = load_image_as_bytes(row['Path'], folders)
        if img_bytes:  # Only append if the image is successfully loaded
            image_data.append(img_bytes)
            labels.append(row['Target'])
            captions.append(row['Caption'])
        else:
            print(f"Image not found: {row['Path']}")

    if not image_data:
        print("No images were loaded. Check the file paths and extensions.")
        return None

    # Create Hugging Face Dataset
    data_dict = {
        'image': image_data,
        'label': labels,
        'caption': captions
    }
    return Dataset.from_dict(data_dict)

# Function to display an image by index
def display_image(dataset, index):
    if 0 <= index < len(dataset):
        image_bytes = dataset[index]['image']
        label = dataset[index]['label']
        caption = dataset[index]['caption']
        
        # Convert bytes back to an image
        image = Image.open(io.BytesIO(image_bytes))
        image.show()
        print(f"Label: {label}")
        print(f"Caption: {caption}")
    else:
        print(f"Invalid index: {index}. Dataset contains {len(dataset)} entries.")

# Process the dataset
dataset = process_dataframe(df, [autism_folder, non_autism_folder])

# Display the first image for verification
if dataset:
    display_image(dataset, 10)  # Display the first image


KeyError: 'Target'

In [82]:
dataset.shape

(1100, 3)

In [61]:
df.shape

(1100, 7)

In [62]:
df.iloc[10]

Age                                                               21.48
SEX                                                                Male
FIQ                                                               128.0
EYE_STATUS_AT_SCAN                                               closed
Target                                                           Autism
Caption               The patient is a male 21-year-old with an FIQ ...
Path                                                         PITT_50012
Name: 10, dtype: object

In [63]:
df.shape

(1100, 7)

In [101]:
# from datasets import DatasetInfo
# from huggingface_hub import HfApi

# # Define the repository name (should be the name of the repository you created)
# repo_name = "multimodal_ASD1"

# # Specify your Hugging Face user
# hf_user = "Bhagya11"

# # Authenticate and create a repository
# api = HfApi()
# repo_url = api.create_repo(repo_id=f"{hf_user}/{repo_name}", exist_ok=True, private=False)

# # Push the dataset
# dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [102]:
# from datasets import Dataset, Features, Value, ClassLabel, Image

# # Define the features for your dataset
# features = Features({
#     "image": Image(),  # Specify the image column as Image type
#     "label": ClassLabel(names=["Autism", "NonAutism"]),  # Define labels
#     "caption": Value("string")  # Caption as a string
# })

# # Recreate your dataset with these features
# dataset = Dataset.from_dict(data_dict, features=features)

# # Push the updated dataset to the Hugging Face Hub
# dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

In [67]:
df

,Age,SEX,FIQ,EYE_STATUS_AT_SCAN,Target,Caption,Path
0,16.77,Male,103.0,closed,Autism,The patient is a 16-year-old male with an FIQ ...,PITT_50002
1,24.45,Male,124.0,closed,Autism,A 24-year-old male with an FIQ score of 124 an...,PITT_50003
2,19.09,Male,113.0,closed,Autism,The 19-year-old male patient with an FIQ score...,PITT_50004
3,13.73,Female,119.0,closed,Autism,The female patient aged 13.73 with an FIQ scor...,PITT_50005
4,13.37,Male,109.0,closed,Autism,The 13-year-old male patient with an FIQ score...,PITT_50006
...,...,...,...,...,...,...,...
1095,35.00,Male,-9999.0,closed,Autism,"The patient is a male, aged 35, diagnosed with...",SBL_51583
1096,49.00,Male,-9999.0,closed,Autism,The patient is a 49-year-old male with an FIQ ...,SBL_51584
1097,27.00,Male,-9999.0,closed,Autism,"The patient, a 27-year-old male with an unrepo...",SBL_51585
1098,29.00,Female,-9999.0,closed,Autism,The patient is a 29-year-old female with an FI...,MAX_MUN_51606


In [83]:
def process_dataframe(df, folders):
    image_data = []
    labels = []
    captions = []

    for _, row in df.iterrows():
        img_bytes = load_image_as_bytes(row['Path'], folders)
        if img_bytes:  # Only append if the image is successfully loaded
            image_data.append(img_bytes)
            labels.append(row['Target'])
            captions.append(row['Caption'])
        else:
            print(f"Image not found: {row['Path']}")

    print(f"Total loaded images: {len(image_data)}")  # Debug

    # Check if all images are loaded
    if len(image_data) != len(df):
        print("Warning: Some images were not loaded.")

    # Create Hugging Face Dataset
    data_dict = {
        'image': image_data,
        'label': labels,
        'caption': captions
    }
    return Dataset.from_dict(data_dict)


In [105]:
# dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")


Map:   0%|          | 0/5 [00:00<?, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

In [106]:
from datasets import load_dataset

dataset = load_dataset(f"{hf_user}/{repo_name}")
print(f"Loaded dataset size: {len(dataset)}")


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5 [00:00<?, ? examples/s]

Loaded dataset size: 1


In [107]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 5
    })
})

In [36]:
df.shape

(1100, 7)

In [39]:
df['Caption'][1]

'A 24-year-old male patient with a FIQ score of 124 has closed eye status at scan.'

In [88]:
df.to_csv('temp.csv')

#### Updated

In [40]:
import os
import io
from PIL import Image
import pandas as pd
from datasets import Dataset, Features, Image as HFImage, Value, ClassLabel
from huggingface_hub import HfApi

# Define the folders
autism_folder = "sMRI_Data/Autism"
non_autism_folder = "sMRI_Data/NonAutism"

# Function to load images and return their paths
def load_image_as_path(file_name, folders):
    for folder in folders:
        for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
            full_path = os.path.join(folder, file_name + ext)
            if os.path.exists(full_path):
                return full_path  # Return the path instead of bytes
    return None

# Function to process the dataset
def process_dataframe(df, folders):
    image_paths = []
    labels = []
    captions = []
    ages = []
    sexes = []
    fiqs = []
    eye_statuses = []

    for _, row in df.iterrows():
        img_path = load_image_as_path(row['Path'], folders)
        if img_path:  # Only append if the image is successfully loaded
            image_paths.append(img_path)
            labels.append(row['Target'])
            captions.append(row['Caption'])
            ages.append(row['Age'])
            sexes.append(row['SEX'])
            fiqs.append(row['FIQ'])
            eye_statuses.append(row['EYE_STATUS_AT_SCAN'])
        else:
            print(f"Image not found: {row['Path']}")

    # Check lengths of all columns
    data_lengths = {
        "image": len(image_paths),
        "label": len(labels),
        "caption": len(captions),
        "age": len(ages),
        "sex": len(sexes),
        "fiq": len(fiqs),
        "eye_status": len(eye_statuses)
    }

    for column, length in data_lengths.items():
        print(f"Length of {column}: {length}")

    # Ensure all columns have the same length
    if len(set(data_lengths.values())) > 1:
        print("Error: Column lengths are inconsistent. Dataset cannot be created.")
        return None

    # Define the dataset features
    features = Features({
        "image": HFImage(),  # Use Image feature for proper display on Hugging Face
        "label": ClassLabel(names=["Autism", "NonAutism"]),  # Define class labels
        "caption": Value("string"),
        "age": Value("int32"),
        "sex": Value("string"),
        "fiq": Value("float32"),
        "eye_status": Value("string")
    })

    # Create Hugging Face Dataset
    data_dict = {
        'image': image_paths,
        'label': labels,
        'caption': captions,
        'age': ages,
        'sex': sexes,
        'fiq': fiqs,
        'eye_status': eye_statuses
    }
    return Dataset.from_dict(data_dict, features=features)

# Function to display an image by index
def display_image(dataset, index):
    if 0 <= index < len(dataset):
        image = dataset[index]['image']  # Load image directly
        label = dataset[index]['label']
        caption = dataset[index]['caption']
        age = dataset[index]['age']
        sex = dataset[index]['sex']
        fiq = dataset[index]['fiq']
        eye_status = dataset[index]['eye_status']

        # Display the image
        image.show()

        # Print metadata
        print(f"Label: {label}")
        print(f"Caption: {caption}")
        print(f"Age: {age}")
        print(f"Sex: {sex}")
        print(f"FIQ: {fiq}")
        print(f"Eye Status: {eye_status}")
    else:
        print(f"Invalid index: {index}. Dataset contains {len(dataset)} entries.")

# Load your DataFrame (assuming it has columns: Path, Target, Caption, Age, SEX, FIQ, EYE_STATUS_AT_SCAN)
# df = pd.read_csv("path_to_your_csv_file.csv")

# Process the dataset
dataset = process_dataframe(df, [autism_folder, non_autism_folder])

# Push the dataset to Hugging Face Hub
if dataset:
    # Define repository details
    repo_name = "ASD_Data"  # Replace with your repository name
    hf_user = "Bhagya11"  # Replace with your Hugging Face username

    # Push dataset to the Hugging Face Hub
    dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")

    # Verify and display the dataset link
    print(f"Dataset uploaded to: https://huggingface.co/datasets/{hf_user}/{repo_name}")


Length of image: 1100
Length of label: 1100
Length of caption: 1100
Length of age: 1100
Length of sex: 1100
Length of fiq: 1100
Length of eye_status: 1100


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Dataset uploaded to: https://huggingface.co/datasets/Bhagya11/ASD_Data


In [92]:
import os
import io
from PIL import Image
import pandas as pd
from datasets import Dataset, Features, Image, Value, ClassLabel
from huggingface_hub import HfApi

# Define the folders
autism_folder = "sMRI_Data/Autism"
non_autism_folder = "sMRI_Data/NonAutism"

# Function to load images and convert them to bytes
def load_image_as_bytes(file_name, folders):
    for folder in folders:
        for ext in ['.jpg', '.png', '.jpeg']:  # Check for common image extensions
            full_path = os.path.join(folder, file_name + ext)
            if os.path.exists(full_path):
                return full_path  # Return the path instead of bytes
    return None

# Function to process the dataset
def process_dataframe(df, folders):
    image_paths = []
    labels = []
    captions = []

    for _, row in df.iterrows():
        img_path = load_image_as_bytes(row['Path'], folders)
        if img_path:  # Only append if the image is successfully loaded
            image_paths.append(img_path)
            labels.append(row['Target'])
            captions.append(row['Caption'])
        else:
            print(f"Image not found: {row['Path']}")

    if not image_paths:
        print("No images were loaded. Check the file paths and extensions.")
        return None

    # Define the dataset features
    features = Features({
        "image": Image(),  # Use Image feature for proper display on Hugging Face
        "label": ClassLabel(names=["Autism", "NonAutism"]),  # Define class labels
        "caption": Value("string")  # Caption as a string
    })

    # Create Hugging Face Dataset
    data_dict = {
        'image': image_paths,
        'label': labels,
        'caption': captions
    }
    return Dataset.from_dict(data_dict, features=features)

# Function to display an image by index
def display_image(dataset, index):
    if 0 <= index < len(dataset):
        image = dataset[index]['image']  # Load image directly
        label = dataset[index]['label']
        caption = dataset[index]['caption']
        
        # Display the image
        image.show()
        print(f"Label: {label}")
        print(f"Caption: {caption}")
    else:
        print(f"Invalid index: {index}. Dataset contains {len(dataset)} entries.")

# Load your DataFrame (assuming it has columns: Path, Target, Caption)
# Replace this with the actual loading of your DataFrame
# df = pd.DataFrame({
#     "Path": ["image1", "image2", "image3"],  # Replace with actual paths (without extensions)
#     "Target": ["Autism", "NonAutism", "Autism"],
#     "Caption": ["Caption 1", "Caption 2", "Caption 3"]
# })

# Process the dataset
dataset = process_dataframe(df, [autism_folder, non_autism_folder])

# Push the dataset to Hugging Face Hub
if dataset:
    # Define repository details
    repo_name = "multimodal_ASD_Final"  # Replace with your repository name
    hf_user = "Bhagya11"  # Replace with your Hugging Face username

    # Push dataset to the Hugging Face Hub
    dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")

    # Verify and display the dataset link
    print(f"Dataset uploaded to: https://huggingface.co/datasets/{hf_user}/{repo_name}")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/1100 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

Dataset uploaded to: https://huggingface.co/datasets/Bhagya11/multimodal_ASD_Final


In [20]:
# from datasets import Dataset, Features, Image, Value, ClassLabel
# from huggingface_hub import HfApi

# # Define the full dataset
# data_dict = {
#     'image': image_paths,  # Your list of 38 image paths
#     'label': labels,       # Your list of 38 labels
#     'caption': captions    # Your list of 38 captions
# }

# # Define the dataset features
# features = Features({
#     'image': Image(),
#     'label': ClassLabel(names=["Autism", "NonAutism"]),
#     'caption': Value("string")
# })

# # Create the dataset
# dataset = Dataset.from_dict(data_dict, features=features)

# # Push the dataset to Hugging Face Hub
# hf_user = "Bhagya11"
# repo_name = "multimodal_ASD1"
# dataset.push_to_hub(repo_id=f"{hf_user}/{repo_name}")


In [21]:
# # Display dataset features
# print(dataset)

# # Display the first example
# print(dataset['train'][0])


In [22]:
from datasets import load_dataset

dataset = load_dataset("Bhagya11/multimodal_ASD1")
print(dataset)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/38 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 38
    })
})


In [28]:
# Access the 'train' dataset
train_dataset = dataset["train"]

# Display the first few rows of the dataset
train_dataset

Dataset({
    features: ['image', 'label', 'caption'],
    num_rows: 38
})

In [29]:
from datasets import load_dataset

# Load the dataset from the Hugging Face Hub
dataset = load_dataset("Bhagya11/multimodal_ASD1")

# Print the structure of the dataset
print(dataset)

# Display the first few rows of the dataset (train, test, or validation split)
# This will display the first instance of each split (train, test, etc.)
print(dataset['train'][0])  # Adjust based on the available splits in the dataset


DatasetDict({
    train: Dataset({
        features: ['image', 'label', 'caption'],
        num_rows: 38
    })
})
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=640x480 at 0x2728C443DF0>, 'label': 0, 'caption': 'The patient is a 55-year-old male with an Intelligence Quotient (IQ) of 126 and targets treatment for Autism.'}


In [100]:
from datasets import load_dataset
import matplotlib.pyplot as plt

# Load the dataset from the Hugging Face Hub
dataset = load_dataset("multimodal_ASD_Final")

# Display the structure of the dataset
print(dataset)

# Get the first data point from the 'train' split
data_point = dataset['train'][0]

# Print the data (label and caption)
print(f"Label: {data_point['label']}")
print(f"Caption: {data_point['caption']}")

# Extract the image data (it's already a PIL Image)
image_data = data_point['image']

# Display the image with its label and caption
plt.imshow(image_data)
plt.title(f"Label: {data_point['label']}\nCaption: {data_point['caption']}")
plt.axis('off')  # Hide axis
plt.show()


DatasetNotFoundError: Dataset 'multimodal_ASD_Final' doesn't exist on the Hub or cannot be accessed.

In [34]:
df

,Age,SEX,FIQ,Target,Path,Caption
0,55.4,Male,126,Autism,CALTECH_51456,The patient is a 55-year-old male with an Inte...
1,22.9,Male,107,Autism,CALTECH_51457,"The patient is 22 years and nine months old, m..."
2,39.2,Male,93,Autism,CALTECH_51458,The patient is a 39-year-old male with an IQ o...
3,22.8,Male,106,Autism,CALTECH_51459,The patient is a 23-year-old male with an Inte...
4,34.6,Female,133,Autism,CALTECH_51460,The patient is a 34.6-year-old female with an ...
5,37.7,Male,99,Autism,CALTECH_51461,The patient is a 37-year-old male with an IQ o...
6,20.2,Female,107,Autism,CALTECH_51462,The patient is a 20.2-year-old female with an ...
7,20.2,Female,102,Autism,CALTECH_51463,The patient is a 20-year-old female with an In...
8,20.9,Male,101,Autism,CALTECH_51464,"'{Age': 20.9, 'SEX': 'Male', 'FIQ': 101, 'Targ..."
9,20.2,Male,96,Autism,CALTECH_51465,The patient is a 20-year-old male with an IQ o...


In [94]:
import pandas as pd

sample = pd.read_parquet("hf://datasets/Bhagya11/multimodal_ASD_Final/data/train-00000-of-00001.parquet")

In [97]:
!pip install pandas openpyxl pillow


Note: you may need to restart the kernel to use updated packages.


In [98]:
import os
from PIL import Image
import pandas as pd
import io
from openpyxl import Workbook
from openpyxl.drawing.image import Image as OpenPyxlImage

# Load the data
sample = pd.read_parquet("hf://datasets/Bhagya11/multimodal_ASD_Final/data/train-00000-of-00001.parquet")

# Create a new Excel workbook
wb = Workbook()
ws = wb.active

# Add headers
ws.append(["Image", "Label", "Caption"])

# Convert image bytes to images and embed them in Excel
for idx, row in sample.iterrows():
    # Extract image bytes and convert to an image object
    image_bytes = row['image']['bytes']
    pil_image = Image.open(io.BytesIO(image_bytes))

    # Save the image temporarily
    temp_image_path = f"temp_image_{idx}.png"
    pil_image.save(temp_image_path)

    # Add the image to the Excel cell
    img = OpenPyxlImage(temp_image_path)
    img.width, img.height = 80, 80  # Resize the image for Excel
    ws.add_image(img, f"A{idx + 2}")  # Place the image in column A (starting from row 2)

    # Add label and caption in the adjacent cells
    ws.cell(row=idx + 2, column=2, value=row['label'])
    ws.cell(row=idx + 2, column=3, value=row['caption'])

# Save the Excel file
excel_output_path = "data_with_images.xlsx"
wb.save(excel_output_path)

# Remove all temporary files after saving the Excel file
for idx in range(len(sample)):
    temp_image_path = f"temp_image_{idx}.png"
    if os.path.exists(temp_image_path):
        os.remove(temp_image_path)

print(f"Excel file with images saved to {excel_output_path}")


Excel file with images saved to data_with_images.xlsx


In [101]:
from datasets import load_dataset

ds = load_dataset("Bhagya11/multimodal_ASD_Final")

README.md:   0%|          | 0.00/425 [00:00<?, ?B/s]

C:\Users\SRMAP\anaconda3\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SRMAP\.cache\huggingface\hub\datasets--Bhagya11--multimodal_ASD_Final. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
C:\Users\SRMAP\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.2

train-00000-of-00001.parquet:   0%|          | 0.00/39.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1100 [00:00<?, ? examples/s]

In [107]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("Bhagya11/multimodal_ASD_Final")

# Access the train split (or any other split) and convert to Pandas
train_df = ds["train"].to_pandas()

# Display the first few rows
print(train_df.head())


                                               image  label  \
0  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0   
1  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0   
2  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0   
3  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0   
4  {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...      0   

                                             caption  
0  The patient is a 16-year-old male with an FIQ ...  
1  A 24-year-old male with an FIQ score of 124 an...  
2  The 19-year-old male patient with an FIQ score...  
3  The female patient aged 13.73 with an FIQ scor...  
4  The 13-year-old male patient with an FIQ score...  
